In [ ]:
import tarfile
import shutil
import random
import os
import cv2
import pathlib
import numpy as np
import pandas as pd

import tensorflow as tf
import matplotlib.pyplot as plt
from numpy.random import seed
from google.colab import drive
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import mixed_precision
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler, EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import ResNet50

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import top_k_accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


# Import Dataset

In [ ]:
%%capture
!tar --extract --verbose --file='/content/gdrive/MyDrive/Progetto Visual/train_balanced.tar'
!unzip '/content/gdrive/MyDrive/Progetto Visual/val_set_degraded.zip' -d '/content/'
!unzip '/content/gdrive/MyDrive/Progetto Visual/val_set.zip'

In [ ]:
csv_train_file = pd.read_csv("/content/gdrive/MyDrive/Progetto Visual/train_balanced_info.csv", dtype=str)
csv_train_file.columns = ['filename', 'label']
parent_dir = "train_set/"
labels = csv_train_file['label']
for label in labels:
    path = os.path.join(parent_dir, label)
    os.makedirs(path, exist_ok=True)    
for _, row in csv_train_file.iterrows():
    label = row['label']
    path = os.path.join(parent_dir, row['filename'])
    img_name = os.path.split(path)[-1]
    new_path = os.path.join(parent_dir, label, img_name)
    print(new_path)
    shutil.copy(path, new_path)

In [ ]:
csv_test_file = pd.read_csv("/content/gdrive/MyDrive/Progetto Visual/val_info.csv", dtype=str)
csv_test_file.columns = ['filename', 'label']
parent_dir = "val_set/"
labels = csv_test_file['label']
for label in labels:
    path = os.path.join(parent_dir, label)
    os.makedirs(path, exist_ok=True)    
for _, row in csv_test_file.iterrows():
    label = row['label']
    path = os.path.join(parent_dir, row['filename'])
    img_name = os.path.split(path)[-1]
    new_path = os.path.join(parent_dir, label, img_name)
    print(new_path)
    shutil.copy(path, new_path)

In [ ]:
train_data_dir = pathlib.Path('train_set/')
seed_train_validation = 42 # must be same for train_ds and val_ds
shuffle_value = True
validation_split = 0.2
batch_size = 256

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_data_dir,
    labels="inferred",
    label_mode="categorical",
    validation_split=validation_split,
    subset="training",
    seed=seed_train_validation,
    image_size=(224,224),
    batch_size=batch_size,
    shuffle=shuffle_value)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_data_dir,
    labels="inferred",
    label_mode="categorical",
    validation_split=validation_split,
    subset="validation",
    seed=seed_train_validation,
    image_size=(224,224),
    batch_size=batch_size,
    shuffle=shuffle_value)

Found 125499 files belonging to 251 classes.
Using 100400 files for training.
Found 125499 files belonging to 251 classes.
Using 25099 files for validation.


In [ ]:
test_data_dir = pathlib.Path('val_set/')
shuffle_value = True

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_data_dir,
    labels="inferred",
    label_mode="categorical",
    subset=None,
    image_size=(224,224),
    batch_size=batch_size,
    shuffle=shuffle_value)

Found 11993 files belonging to 251 classes.


In [ ]:
def preprocess(images, labels):
  #return tf.keras.applications.resnet.preprocess_input(images), labels
  return tf.keras.applications.resnet50.preprocess_input(images), labels

train_ds = train_ds.map(preprocess)
val_ds = val_ds.map(preprocess)
test_ds = test_ds.map(preprocess)

# Creazione Modello

In [ ]:
baseModel = ResNet50(weights='imagenet', include_top=True, input_shape= (224, 224, 3))
baseModel.summary()

102967424/102967424 [==============================] - 1s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                        

In [ ]:
# Freeze layer del baseModel
for layer in baseModel.layers:
	layer.trainable = False

model = Model(inputs=baseModel.input, outputs=baseModel.get_layer('avg_pool').output)
last_layer = baseModel.get_layer('avg_pool').output
flatten_layer = Flatten()(last_layer)
classification_layer = Dense(251, activation='softmax')(flatten_layer)
model = Model(inputs=model.input, outputs=classification_layer)
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [ ]:
def decay_schedule(epoch, lr):
    # decay by 0.1 every 5 epochs; 
    if (epoch % 5 == 0) and (epoch != 0):
        lr = lr * 0.1
    return lr
lr_scheduler = LearningRateScheduler(decay_schedule)

early_stop = EarlyStopping(monitor='val_loss', patience=50, verbose=1, restore_best_weights=True) 
best_model = ModelCheckpoint(checkpoint_filepath, verbose=1, monitor='val_loss', mode='min', save_best_only=True, save_weights_only=True)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=3)])
history = model.fit(train_ds, epochs=10, verbose=1, validation_data=val_ds, callbacks=[lr_scheduler, early_stop])

Epoch 1/10
393/393 [==============================] - 512s 1s/step - loss: 3.3385 - accuracy: 0.2966 - top_k_categorical_accuracy: 0.4569 - val_loss: 2.7756 - val_accuracy: 0.3858 - val_top_k_categorical_accuracy: 0.5698 - lr: 0.0010
Epoch 2/10
393/393 [==============================] - 489s 1s/step - loss: 2.3972 - accuracy: 0.4545 - top_k_categorical_accuracy: 0.6378 - val_loss: 2.6313 - val_accuracy: 0.4154 - val_top_k_categorical_accuracy: 0.5994 - lr: 0.0010
Epoch 3/10
393/393 [==============================] - 529s 1s/step - loss: 2.0776 - accuracy: 0.5189 - top_k_categorical_accuracy: 0.6966 - val_loss: 2.5533 - val_accuracy: 0.4300 - val_top_k_categorical_accuracy: 0.6162 - lr: 0.0010
Epoch 4/10
393/393 [==============================] - 481s 1s/step - loss: 1.8643 - accuracy: 0.5610 - top_k_categorical_accuracy: 0.7350 - val_loss: 2.5187 - val_accuracy: 0.4360 - val_top_k_categorical_accuracy: 0.6258 - lr: 0.0010
Epoch 5/10
393/393 [==============================] - 481s 1s/st

In [ ]:
score = model.evaluate(test_ds, batch_size=512)
print('test loss, test acc, test top-3 acc:', score)

47/47 [==============================] - 54s 1s/step - loss: 2.0826 - accuracy: 0.4999 - top_k_categorical_accuracy: 0.6962
test loss, test acc, test top-3 acc: [2.0825564861297607, 0.49987491965293884, 0.6962394714355469]


# Test Degradation

In [ ]:
%%capture
csv_test_degraded_file = pd.read_csv("/content/gdrive/MyDrive/Progetto Visual/val_info.csv", dtype=str)
csv_test_degraded_file.columns = ['filename', 'label']
parent_dir = "val_set_degraded/"
labels = csv_test_degraded_file['label']
for label in labels:
    path = os.path.join(parent_dir, label)
    os.makedirs(path, exist_ok=True)    
for _, row in csv_test_file.iterrows():
    label = row['label']
    path = os.path.join(parent_dir, row['filename'])
    img_name = os.path.split(path)[-1]
    new_path = os.path.join(parent_dir, label, img_name)
    print(new_path)
    shutil.copy(path, new_path)

In [ ]:
test_degraded_data_dir = pathlib.Path('val_set_degraded/')
shuffle_value = True
test_degraded_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_degraded_data_dir,
    labels="inferred",
    label_mode="categorical",
    subset=None,
    image_size=(224,224),
    batch_size=batch_size,
    shuffle=shuffle_value)

Found 11993 files belonging to 251 classes.


In [ ]:
score = model.evaluate(test_degraded_ds, batch_size=512)
print('test loss, test acc, test top-3 acc:', score)

47/47 [==============================] - 43s 857ms/step - loss: 8.3496 - accuracy: 0.0351 - top_k_categorical_accuracy: 0.0738
test loss, test acc, test top-3 acc: [8.34962272644043, 0.03510380908846855, 0.07379304617643356]


In [ ]:
#model.save('/content/gdrive/MyDrive/Progetto Visual/Resnet50_augmented.h5')


In [ ]:
#Resnet50 Addestrato con Training set bilanciato

# Test Set & Test Set degradato
#test loss, test acc, test top-3 acc: [2.0825564861297607, 0.49987491965293884, 0.6962394714355469]
#test loss, test acc, test top-3 acc: [8.34962272644043, 0.03510380908846855, 0.07379304617643356]
